# Extracting desired business reviews
In the last notebook we achieved two things. Firstly, we identified the business categories (Italian/Pizza) that our campaign was going to target. Secondly, we identified business IDs associated with those categories. Knowing the business IDs allows us to select just the reviews of interest from the reviews file. Before you go rushing into reading the review data into one big dataframe to then filter just those IDs of interest, as a data scientist you will have first got an idea of the size of the file. The review file is pretty big (nearly six million lines). Unless you have a very good computer with lots of RAM, or you particularly want to fire up your system/memory monitor and watch the free memory fall further and further until your computer seizes, you should be thinking in terms of how to read in only the lines of interest. This way, you only consume the minimum RAM necessary. Of course, in general, even then you should be making some sort of calculation as to whether even this would be within the capabilities of your hardware. In this case, it should be doable for most modernish computers. The task of this notebook is to parse the review data file and save the (much smaller) subset of interest.

In [1]:
# import the required libraries here
import pandas as pd
import numpy as np

## List of required business IDs
First we need to read in the list of business IDs we obtained previously.

In [2]:
# task: read in our previously created file that contains the business IDs of interest
# one line
%cd C:\Users\Pranati\Documents\books\SpringboardPracticeProjects\yelp_dataset
businesses = pd.read_csv('yelp_business_list.csv')

C:\Users\Pranati\Documents\books\SpringboardPracticeProjects\yelp_dataset


It's a good idea to have a quick check first.

In [3]:
businesses.head()

,business_id,review_count,stars,state
0,PZ-LZzSlhSe9utkQYU8pFg,40,4.0,NV
1,bJP4l_BGq2CudEu0m-wNjg,3,2.0,NV
2,jwHtNvHBPafiAwM92jM8yA,3,4.0,ON
3,111d13yLgIdqvpZekgVnbQ,5,3.0,ON
4,dUffgo9Lh_Vk9TLuFR5ywg,563,3.5,AZ


We want to grab just the business_id column as a list. It's worth also then double checking it matches the business_id column above.

In [4]:
business_ids = businesses['business_id'].values
print(business_ids[:5])

['PZ-LZzSlhSe9utkQYU8pFg' 'bJP4l_BGq2CudEu0m-wNjg'
 'jwHtNvHBPafiAwM92jM8yA' '111d13yLgIdqvpZekgVnbQ'
 'dUffgo9Lh_Vk9TLuFR5ywg']


## Filtering a large data file in chunks
The review data file comprises some nearly 6 million lines. We don't want all of those lines. Our general approach will be to read through the file and check the business_id in each row. If this ID is contained in our list of wanted IDs, we will keep that row. It will be fastest, but most expensive for memory, to read the entire file from disk in one go and then filter the rows in memory. At the other extreme it will be most memory efficient to read in one record at a time, check whether we want it or not (and discard if not), but slowest. We will strike a balance where we choose a sufficiently large chunksize that still fits easily in memory but whereby we don't need to read in too many chunks.

In [24]:
# For convenience here, again we are accessing the data in the working directory that contains our notebooks.
# Normal best practise is to keep your data separate, but this keeps things simple here.
# If you're comfortable specifying a filepath to files outside of this working directory, then feel free to do so.
# task: create a reader object for the review json file
# Hint: use lines=True as before but add the chunksize=100000 parameter
# one line of code here
review_reader = pd.read_json('review.json',lines=True,chunksize=100000)

We use the time magic for the next cell, out of interest, to time how long it takes. If you have the time and are curious, you're welcome to play around with the chunksize and monitor your system memory use and the time this cell takes, but _caveat emptor_! I suggest you save your notebook before running this cell, just in case.

In [25]:
# task: process the file one chunk at a time,
# filter that chunk for rows with a business_id in business_ids
# You can either do this within in a loop, having initialized an empty list,
# or using a more pythonic list comprehension
reviews = [review.loc[review['business_id'].apply(lambda x: x in business_ids)] for review in review_reader]
# (this took some 24 minutes on my old i7)

All being well, you now have the desired reviews read in. But we don't yet have them in a convenient DataFrame:

In [26]:
type(reviews)

list

In [27]:
len(reviews)

67

Convert your reviews into a DataFrame now. _Hint: you may find pandas concat method useful here._

In [28]:
# task convert your reviews into a DataFrame
# one line of code here
reviews = pd.concat(reviews)

In [29]:
reviews.shape

(145508, 9)

In [30]:
reviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
5,eU_713ec6fTGNO4BegRaww,0,2013-01-20 13:25:59,0,fdiNeiN_hoCxCMy2wTRW9g,4,I'll be the first to admit that I was not exci...,0,w31MKYsNFMrjhWxxAb5wIw
19,y-Iw6dZflNix4BdwIyTNGA,0,2014-06-27 21:19:23,0,4bUyL7lzoWzDZaJETAKREg,3,Good selection of classes of beers and mains. ...,0,_N7Ndn29bpll_961oPeEfw
27,Gyrez6K8f1AyR7dzW9fvAw,1,2013-12-28 22:28:08,0,qm97yMwREr7BKkexlwLFbg,5,their pettuccine was fresh-made in the morning...,0,TTRVdTXKcq-xn6-1IWbwEw
96,Gyrez6K8f1AyR7dzW9fvAw,1,2017-01-02 03:48:03,1,l7KB334CXThAcmZPOvF1zw,2,"Normally, I give a restaurant at least 3 stars...",3,N6zhXHPIrCXommDEg0Rw3g
103,ZLuWpmvO41w2bSwufgfUlQ,3,2010-08-20 22:38:10,0,XRCS5zr29DOxlCXKKKwwAw,5,This is my first yelp. This restaurant was so...,1,2ROluW4TNFbkO3qLtihhaQ


## Save the output
Having done all this hard work filtering our review file, and documenting the process in this notebook, we need to save the output. Go ahead and save the reviews DataFrame to a csv called 'reviews_filtered.csv'.

In [31]:
# task: save the DataFrame to the specified file now.
# don't forget to use index=False
reviews.to_csv('reviews_filtered.csv',index = False)

If you inspect the file listing for the original json file and our new csv file, you should see we've gone from around 4.4 GB down to 325 MB. This is much more manageable!

# Summary
Great work! So far in our project you've seen how to break a problem down and identify relevant data, use data to gain insight and make a decision, and then leverage that knowledge to extract a desired subset of data from an otherwise unmanageable larger file. In the next notebook we'll dive into this data.